In [6]:
import pandas as pd

file_path = "/content/credit_card_transactions.csv"
df = pd.read_csv(file_path)

print("Dataset Info:")
print(df.info())
print("\nFirst 5 rows:")
print(df.head())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207860 entries, 0 to 207859
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             207860 non-null  int64  
 1   trans_date_trans_time  207860 non-null  object 
 2   cc_num                 207860 non-null  int64  
 3   merchant               207860 non-null  object 
 4   category               207860 non-null  object 
 5   amt                    207860 non-null  float64
 6   first                  207860 non-null  object 
 7   last                   207860 non-null  object 
 8   gender                 207860 non-null  object 
 9   street                 207860 non-null  object 
 10  city                   207860 non-null  object 
 11  state                  207860 non-null  object 
 12  zip                    207859 non-null  float64
 13  lat                    207859 non-null  float64
 14  long                  

In [7]:
user_card_count = df.groupby(['first', 'last', 'dob'])['cc_num'].nunique().reset_index()
user_card_count = user_card_count.rename(columns={'cc_num': 'card_count'})

multi_card_users = user_card_count[user_card_count['card_count'] > 1]
print("Users with multiple cards:")
print(multi_card_users)


Users with multiple cards:
Empty DataFrame
Columns: [first, last, dob, card_count]
Index: []


In [9]:
user_card_check = df.groupby(['first', 'last', 'dob'])['cc_num'].nunique()
print("Unique card counts per user:")
print(user_card_check.describe())


Unique card counts per user:
count    932.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: cc_num, dtype: float64


In [10]:
transactions_per_card = df['cc_num'].value_counts()
print("Transaction distribution across cards:")
print(transactions_per_card.describe())


Transaction distribution across cards:
count    932.000000
mean     223.025751
std      124.216442
min        7.000000
25%       96.000000
50%      227.500000
75%      318.000000
max      552.000000
Name: count, dtype: float64


In [15]:
reward_structure = {
    'entertainment': 5,    # x points per $1 spent
    'grocery_pos': 3,
    'misc_net': 2,
    'gas_transport': 4,
    'misc_pos': 1,
    'shopping_net': 3,
    'shopping_pos': 2,
    'travel': 6
}

# Assign reward points
df['reward_points'] = df['category'].map(reward_structure) * df['amt']

# Preview
print("Sample transactions with calculated reward points:")
print(df[['category', 'amt', 'reward_points']].head())


Sample transactions with calculated reward points:
        category     amt  reward_points
0       misc_net    4.97           9.94
1    grocery_pos  107.23         321.69
2  entertainment  220.11        1100.55
3  gas_transport   45.00         180.00
4       misc_pos   41.96          41.96


In [16]:
user_rewards = df.groupby(['first', 'last', 'cc_num']).agg(total_spent=('amt', 'sum'),total_rewards=('reward_points', 'sum')).reset_index()

print("Total spending and rewards per user:")
print(user_rewards.head())


Total spending and rewards per user:
   first     last               cc_num  total_spent  total_rewards
0  Aaron   Murray      376028110684021     37849.45      104984.90
1  Aaron     Pena  4958589671582726883     15213.08       21657.07
2  Aaron   Rogers     4260059589824237      4023.31        6086.54
3  Aaron  Stewart     4228411452607671      2347.14        3506.88
4   Adam   Keller        4917226033950      9787.92       22543.82


In [17]:
category_analysis = df.groupby('category').agg(total_spent=('amt', 'sum'), total_rewards=('reward_points', 'sum')).reset_index()

category_analysis = category_analysis.sort_values(by='total_rewards', ascending=False)

print("Spending and reward points by category:")
print(category_analysis)


Spending and reward points by category:
          category  total_spent  total_rewards
4      grocery_pos   2340174.53     7020523.59
2    gas_transport   1340621.67     5362486.68
0    entertainment    970919.95     4854599.75
13          travel    764376.05     4586256.30
11    shopping_net   1450443.90     4351331.70
12    shopping_pos   1479479.81     2958959.62
8         misc_net    852972.07     1705944.14
9         misc_pos    771342.76      771342.76
1      food_dining    753322.29           0.00
3      grocery_net    391610.69           0.00
5   health_fitness    759654.00           0.00
6             home   1145769.91           0.00
7        kids_pets   1035992.94           0.00
10   personal_care    693801.33           0.00


In [18]:
user_rewards['reward_efficiency'] = user_rewards['total_rewards'] / user_rewards['total_spent']
print("User reward efficiency summary:")
print(user_rewards[['first', 'last', 'cc_num', 'reward_efficiency']].head())


User reward efficiency summary:
   first     last               cc_num  reward_efficiency
0  Aaron   Murray      376028110684021           2.773750
1  Aaron     Pena  4958589671582726883           1.423582
2  Aaron   Rogers     4260059589824237           1.512819
3  Aaron  Stewart     4228411452607671           1.494108
4   Adam   Keller        4917226033950           2.303229


In [20]:
user_category = df.groupby(['first', 'last', 'cc_num', 'category']).agg(
    category_spent=('amt', 'sum'),
    category_rewards=('reward_points', 'sum')
).reset_index()

print("Sample user-category spending and rewards:")
print(user_category.head())


Sample user-category spending and rewards:
   first    last           cc_num       category  category_spent  \
0  Aaron  Murray  376028110684021  entertainment         1980.17   
1  Aaron  Murray  376028110684021    food_dining         1805.59   
2  Aaron  Murray  376028110684021  gas_transport         1263.58   
3  Aaron  Murray  376028110684021    grocery_net          936.38   
4  Aaron  Murray  376028110684021    grocery_pos         4161.41   

   category_rewards  
0           9900.85  
1              0.00  
2           5054.32  
3              0.00  
4          12484.23  


In [21]:
final_dataset = user_category.copy()

final_dataset['reward_efficiency'] = final_dataset['category_rewards'] / final_dataset['category_spent']

final_dataset['total_spent'] = final_dataset.groupby(['first', 'last', 'cc_num'])['category_spent'].transform('sum')
final_dataset['total_rewards'] = final_dataset.groupby(['first', 'last', 'cc_num'])['category_rewards'].transform('sum')
final_dataset = final_dataset.dropna().query('category_spent > 0 and category_rewards > 0')

print(final_dataset.info())
print(final_dataset.head())

final_dataset.to_csv("cleaned_dataset_for_model.csv", index=False)
print("Final dataset saved as 'cleaned_dataset_for_model.csv'")


<class 'pandas.core.frame.DataFrame'>
Index: 7346 entries, 0 to 12716
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   first              7346 non-null   object 
 1   last               7346 non-null   object 
 2   cc_num             7346 non-null   int64  
 3   category           7346 non-null   object 
 4   category_spent     7346 non-null   float64
 5   category_rewards   7346 non-null   float64
 6   reward_efficiency  7346 non-null   float64
 7   total_spent        7346 non-null   float64
 8   total_rewards      7346 non-null   float64
dtypes: float64(5), int64(1), object(3)
memory usage: 573.9+ KB
None
   first    last           cc_num       category  category_spent  \
0  Aaron  Murray  376028110684021  entertainment         1980.17   
2  Aaron  Murray  376028110684021  gas_transport         1263.58   
4  Aaron  Murray  376028110684021    grocery_pos         4161.41   
8  Aaron  Murray  37602811068

In [22]:
from google.colab import files
files.download("cleaned_dataset_for_model.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
n=pd.read_csv("/content/cleaned_dataset_for_model.csv")
dfn=pd.DataFrame(n)
dfn

,first,last,cc_num,category,category_spent,category_rewards,reward_efficiency,total_spent,total_rewards
0,Aaron,Murray,376028110684021,entertainment,2056.16,10280.80,5.0,38126.14,105752.87
1,Aaron,Murray,376028110684021,gas_transport,1263.58,5054.32,4.0,38126.14,105752.87
2,Aaron,Murray,376028110684021,grocery_pos,4290.75,12872.25,3.0,38126.14,105752.87
3,Aaron,Murray,376028110684021,misc_net,2545.19,5090.38,2.0,38126.14,105752.87
4,Aaron,Murray,376028110684021,misc_pos,1134.09,1134.09,1.0,38126.14,105752.87
...,...,...,...,...,...,...,...,...,...
7345,Zachary,Boone,374821819075109,misc_net,27.50,55.00,2.0,5469.68,7182.27
7346,Zachary,Boone,374821819075109,misc_pos,750.23,750.23,1.0,5469.68,7182.27
7347,Zachary,Boone,374821819075109,shopping_net,229.06,687.18,3.0,5469.68,7182.27
7348,Zachary,Boone,374821819075109,shopping_pos,147.96,295.92,2.0,5469.68,7182.27


In [19]:
column_names_n=dfn.columns
print(column_names_n)

Index(['first', 'last', 'cc_num', 'category', 'category_spent',
       'category_rewards', 'reward_efficiency', 'total_spent',
       'total_rewards'],
      dtype='object')
